In [1]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 4.3 MB/s eta 0:00:00


In [1]:
import pandas as pd
import tldextract

In [2]:
import torch
import torch.nn.functional as F
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model + processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
import sys
import ast

conf = dict()

def get_label_new(sentence, images):
    max_len = 77  # max_position_embeddings for CLIP

    # Prepare texts: sentence + labels
    texts = [sentence] + conf['labels']

    # Use clip_processor with truncation and max_length
    text_inputs = clip_processor(
        text=texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_len
    )

    if images:  # images present
        image_inputs = clip_processor(
            images=images,
            return_tensors="pt"
        )

        with torch.no_grad():
            text_outputs = clip_model.get_text_features(**text_inputs)
            image_outputs = clip_model.get_image_features(**image_inputs)

        image_emb = image_outputs.mean(dim=0, keepdim=True)
        sentence_emb = text_outputs[0:1]
        label_embs = text_outputs[1:]

        combined_emb = (sentence_emb + image_emb) / 2

    else:  # only text
        with torch.no_grad():
            text_outputs = clip_model.get_text_features(**text_inputs)
        combined_emb = text_outputs[0:1]
        label_embs = text_outputs[1:]

    similarities = F.cosine_similarity(combined_emb, label_embs)
    closest = similarities.argsort(descending=True)

    result = f"{conf['labels'][closest[0]]}:{similarities[closest[0]]:.3f}, " \
             f"{conf['labels'][closest[1]]}:{similarities[closest[1]]:.3f}"

    return result

labels = ["sports", "politics", "technology"]
conf['labels'] = labels

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/bert/Spider1/crawler_1747902094.csv')

"""
sentence = "Politics"
images = ['/content/drive/MyDrive/bert/Spider1/imgs/42801768_2..png', '/content/drive/MyDrive/bert/Spider1/imgs/38503757_1..png']

images = [Image.open(img) for img in images]

prediction = get_label_new(sentence, images)
print("works:", prediction)

sys.exit(0)
"""

for i in range(df.shape[0]):
  print(f'i = {i}')
  sentence = df.iloc[i]['post_text']
  images_names = df.iloc[i]['downloaded_imgs']
  images_names = ast.literal_eval(images_names)

  images_paths = ["/content/drive/MyDrive/bert/Spider1/imgs/"+name+"..png" for name in images_names]

  valid_images = []
  for path in images_paths:
      try:
          img = Image.open(path)
          valid_images.append(img)
      except:
          print(f"Skipped file {path}")

  prediction = get_label_new(sentence, valid_images)
  df.loc[i, 'label'] = prediction

  # print(prediction)  # Example: "sports:0.91, politics:0.45"

df.to_csv('images_and_sentence_prediction.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
i = 0
i = 1
i = 2


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
i = 30
i = 31
i = 32
i = 33
i = 34
i = 35
i = 36
i = 37
i = 38
i = 39
i = 40
i = 41
i = 42
i = 43
i = 44
i = 45
i = 46


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


i = 47
i = 48
i = 49
i = 50
i = 51
i = 52
i = 53
i = 54
i = 55


...

i = 250
i = 251
i = 252
i = 253
Skipped file /content/drive/MyDrive/bert/Spider1/imgs/22410815_4..png
Skipped file /content/drive/MyDrive/bert/Spider1/imgs/22410815_5..png
Skipped file /content/drive/MyDrive/bert/Spider1/imgs/22410815_7..png
i = 254
i = 255
i = 256
i = 257
i = 258
i = 259


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


i = 260
i = 261
Skipped file /content/drive/MyDrive/bert/Spider1/imgs/58955238_4..png
Skipped file /content/drive/MyDrive/bert/Spider1/imgs/58955238_8..png
Skipped file /content/drive/MyDrive/bert/Spider1/imgs/58955238_22..png
i = 262
i = 263
i = 264
i = 265


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


i = 266
i = 267
i = 268


In [3]:
tecknology_domain = [
"addtoany",
"adobe",
"computerhope",
"computerlanguage",
"eurid",
"google",
"mediawiki",
"merlot",
"oercommons",
"pc"]

bussiness_domain=[
"hbr",
"philadelphiabusinesslist"]

politics_domain=[
"newpol"]

domain_history=[
"historic-uk",
"newworldencyclopedia",
"worldhistory",
"ushistory"]

# loc missouri reddit reimagine-education twitter pod sharpened ox slj         tutorful tutorhouse     wisc

total = 0
total_urls = 0
tech_match = 0
tech_miss_match = 0
label_cnt = 0
n_a = 0

In [ ]:
csv_file = "/content/drive/MyDrive/crawler/Spider1/results.csv"
df = pd.read_csv(csv_file,index_col=False)
for i in range(0,len(df.index)):
  domain = tldextract.extract(df.loc[i,"link"]).domain
  # print(domain)
  total_urls = total_urls +1
  if df.loc[i,"processed"]== "Yes":
    total = total +1
    if domain in tecknology_domain and df.loc[i,"label"].split(':')[0] == "technology":
      label_cnt = label_cnt +1
      tech_match = tech_match + 1
      df.loc[i,"true_label"]="technology"
    elif domain in tecknology_domain and df.loc[i,"label"].split(':')[0] != "technology":
      label_cnt = label_cnt +1
      tech_miss_match = tech_miss_match +1
      df.loc[i,"true_label"]="technology"
    elif domain not in tecknology_domain:
      n_a = n_a + 1
      # print(f"N/A -- {domain}")


print(f"results: total = {total} label:{label_cnt}, tech_match = {tech_match} , tech_miss_match = {tech_miss_match}, n_a = {n_a}")
print(f"total_urls: {total_urls} ")

results: total = 440 label:82, tech_match = 82 , tech_miss_match = 0, n_a = 82
total_urls: 440 


In [ ]:
csv_file = "/content/drive/MyDrive/crawler/Spider1/results.csv"
df = pd.read_csv(csv_file,index_col=False)
for i in range(0,len(df.index)):
  domain = tldextract.extract(df.loc[i,"link"]).domain
  # print(domain)
  total_urls = total_urls +1
  if df.loc[i,"processed"]== "Yes":
    total = total +1
    if domain in domain_history and df.loc[i,"label"].split(':')[0] == "history":
      label_cnt = label_cnt +1
      tech_match = tech_match + 1
      df.loc[i,"true_label"]="history"
    elif domain in domain_history and df.loc[i,"label"].split(':')[0] != "history":
      label_cnt = label_cnt +1
      tech_miss_match = tech_miss_match +1
      df.loc[i,"true_label"]="history"
    elif domain not in domain_history:
      n_a = n_a + 1
      # print(f"N/A -- {domain}")


print(f"results: total = {total} label:{label_cnt}, tech_match = {tech_match} , tech_miss_match = {tech_miss_match}, n_a = {n_a}")

results: total = 609 label:244, tech_match = 115 , tech_miss_match = 129, n_a = 365


In [ ]:
csv_file = "/content/drive/MyDrive/crawler/Spider1/results.csv"
df = pd.read_csv(csv_file,index_col=False)
for i in range(0,len(df.index)):
  domain = tldextract.extract(df.loc[i,"link"]).domain
  # print(domain)
  total_urls = total_urls +1
  if df.loc[i,"processed"]== "Yes":
    total = total +1
    if domain in bussiness_domain and df.loc[i,"label"].split(':')[0] == "business":
      label_cnt = label_cnt +1
      tech_match = tech_match + 1
      df.loc[i,"true_label"]="business"
    elif domain in bussiness_domain and df.loc[i,"label"].split(':')[0] != "business":
      label_cnt = label_cnt +1
      tech_miss_match = tech_miss_match +1
      df.loc[i,"true_label"]="business"
    elif domain not in bussiness_domain:
      n_a = n_a + 1
      # print(f"N/A -- {domain}")


print(f"results: total = {total} label:{label_cnt}, tech_match = {tech_match} , tech_miss_match = {tech_miss_match}, n_a = {n_a}")

results: total = 609 label:61, tech_match = 58 , tech_miss_match = 3, n_a = 548


In [ ]:
csv_file = "/content/drive/MyDrive/crawler/Spider1/results.csv"
df = pd.read_csv(csv_file,index_col=False)
for i in range(0,len(df.index)):
  domain = tldextract.extract(df.loc[i,"link"]).domain
  # print(domain)
  total_urls = total_urls +1
  if df.loc[i,"processed"]== "Yes":
    total = total +1
    if domain in politics_domain and df.loc[i,"label"].split(':')[0] == "politics":
      label_cnt = label_cnt +1
      tech_match = tech_match + 1
      df.loc[i,"true_label"]="politics"
    elif domain in politics_domain and df.loc[i,"label"].split(':')[0] != "politics":
      label_cnt = label_cnt +1
      tech_miss_match = tech_miss_match +1
      df.loc[i,"true_label"]="politics"
    elif domain not in politics_domain:
      n_a = n_a + 1
      # print(f"N/A -- {domain}")


print(f"results: total = {total} label:{label_cnt}, tech_match = {tech_match} , tech_miss_match = {tech_miss_match}, n_a = {n_a}")

results: total = 609 label:51, tech_match = 38 , tech_miss_match = 13, n_a = 558


In [ ]:
with open(csv_file, 'w') as f:
  df.to_csv(f,index=False)